### Setup Asyncio 


In [1]:
import nest_asyncio
nest_asyncio.apply()

### Set up Qdrant vector DB

In [1]:
import qdrant_client

collection_name="chat_with_docs"

client = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)

### Read the documents

In [2]:
from llama_index.core import SimpleDirectoryReader
input_dir_path = './docs'
loader = SimpleDirectoryReader(input_dir=input_dir_path, required_exts=['.pdf'], recursive=True)

In [3]:
docs = loader.load_data()

In [5]:
type(docs), len(docs)

(list, 32)

In [6]:
docs[0]

Document(id_='202660ab-4cff-48ee-a69e-594e98202d1c', embedding=None, metadata={'page_label': '1', 'file_name': 'dspy.pdf', 'file_path': '/home/yuvidh/work/Projects/Langchain_Projects/05_RAG_from_scratch/docs/dspy.pdf', 'file_type': 'application/pdf', 'file_size': 460814, 'creation_date': '2025-01-03', 'last_modified_date': '2024-11-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Preprint\nDSP Y: C OMPILING DECLARATIVE LANGUAGE\nMODEL CALLS INTO SELF -IMPROVING PIPELINES\nOmar Khattab,1 Arnav Singhvi,2\nParidhi Maheshwari,4 Zhiyuan Zhang,1\nKeshav Santhanam,1 Sri Vardhamanan,6 Saiful Haq,6\nAshutosh Sharma,6 Thomas T. Joshi,7 Hanna Mo

### Indexing Data 

In [10]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, ServiceContext, StorageContext

def create_index(documents):
    vector_store = QdrantVectorStore(client=client, collection_name=collection_name)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
    return index



### Loading Embedding Model & Indexing the Data

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5",
                                   trust_remote_code=True)

# Ensuring same model is used throughout the rag pipeline
Settings.embed_model = embed_model

# Convert each document into an embedding using the embed model
index = create_index(docs)

### Load the LLM 

In [17]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model = 'llama3.2:1b', request_timeout=120.0, base_url="http://172.18.176.1:11434")
Settings.llm = llm

### Define the Prompt Template 

In [18]:
from llama_index.core import PromptTemplate

template = """Context information is below:
              ---------------------
              {context_str}
              ---------------------
              Given the context information above I want you to think
              step by step to answer the query in a crisp manner,
              incase you don't know the answer say 'I don't know!'

              Query: {query_str}

              Answer:"""

qa_prompt_tmpl = PromptTemplate(template)

### Reranking

In [19]:
from llama_index.core.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2",
    top_n=3
)

### Querying the Document

In [20]:
query_engine = index.as_query_engine(similarity_top_k=10,
                                     node_postprocessors=[rerank])

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

response = query_engine.query("What exactly is DSPy?")

In [25]:
from IPython.display import Markdown, display
display(Markdown(str(response)))

DSPy stands for "Dependently Specified Programming Yields", and it's a programming model that abstracts prompting techniques into parameterized declarative modules. This allows developers to define interfaces for their natural language processing (NLP) systems using natural language signatures, which can then be used as prompts or instructions for the LMs (Large Language Models).